# Average Metrics Analysis

This .ipynb runs the averageMetrics.py script, and generates multiple analysis plots and charts. 1) P/R per class, per transform, as a table for each model. 2) P/R difference per class, per transform as a table. 3) Multi-Bar Chart per model P/R difference per transform. 4) Per class, Per transform precision and recall averaged over every model (Box plot). 5) Per class, per model performance averaged over all transforms (Box plot). 6) Precision/Recall performance per class, per model, averaged based on the "categories" of transforms (Value/Spatial, sub-categories of each) 7) Perhaps performance per supercategory.

In [106]:
import os
import ast
import subprocess
from multiprocessing import Process, Queue
from multiprocessing import Pool
import glob
import json
import pandas as pd
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
from pycocotools.coco import COCO
import numpy as np
import time
import matplotlib.pyplot as plt

In [107]:
dataDir='../data/inputs'
dataType='val2017'
annFile = '%s/annotations/%s_%s.json'%(dataDir,'instances',dataType)
annType = 'bbox'
cocoGt=COCO(annFile)
cats = cocoGt.loadCats(cocoGt.getCatIds())
nms=[cat['name'] for cat in cats]

transformNames = [
 'None',
 'gaussianblur_1',
 'gaussianblur_10',
 'gaussianblur_20',
 'superpixels_0p1',
 'superpixels_0p5',
 'superpixels_0p85',
 'colorspace_25',
 'colorspace_50',
 'averageblur_5_11',
 'medianblur_1',
 'sharpen_0',
 'sharpen_1',
 'sharpen_2',
 'addintensity_-80',
 'addintensity_80',
 'elementrandomintensity_1',
 'multiplyintensity_0p25',
 'multiplyintensity_2',
 'contrastnormalization_0',
 'contrastnormalization_1',
 'contrastnormalization_2',
 'elastic_1',
 'scaled_1p25',
 'scaled_0p75',
 'scaled_0p5',
 'scaled_(1p25, 1p0)',
 'scaled_(0p75, 1p0)',
 'scaled_(1p0, 1p25)',
 'scaled_(1p0, 0p75)',
 'translate_(0p1, 0p1)',
 'translate_(0p1, -0p1)',
 'translate_(-0p1, 0p1)',
 'translate_(-0p1, -0p1)',
 'translate_(0p1, 0)',
 'translate_(-0p1, 0)',
 'translate_(0, 0p1)',
 'translate_(0, -0p1)',
 'rotated_3',
 'rotated_5',
 'rotated_10',
 'rotated_45',
 'rotated_60',
 'rotated_90',
 'flipH',
 'flipV',
 'dropout',
 ]
modelNames = ['e2e_faster_rcnn_R-50-C4_2x',
'e2e_faster_rcnn_R-50-FPN_2x',
'e2e_faster_rcnn_R-101-FPN_2x',
'e2e_faster_rcnn_X-101-64x4d-FPN_2x',
'e2e_mask_rcnn_R-50-C4_2x',
'e2e_mask_rcnn_R-50-FPN_2x',
'e2e_mask_rcnn_R-101-FPN_2x',
'retinanet_R-50-FPN_2x',
'retinanet_R-101-FPN_2x',
'retinanet_X-101-64x4d-FPN_2x']

loading annotations into memory...
Done (t=0.70s)
creating index...
index created!


### Read metrics for each model and transform, and parse them into pandas data frames.

In [115]:
metricsColumns = ['model','transform','class','precision','recall','accuracy','TP','FP','FN']
metricsList = []
metrics = pd.DataFrame([], columns=metricsColumns)
regenerate=False


def f(tup):
    print('%s,%s,%s running...'%(tup[1], tup[2], str(tup[0])))
    result = subprocess.check_output(args=["python", "averageMetrics.py", "-c", str(tup[0]), '-r',
                             'basic', tup[1], tup[2]])
    output = result.decode("utf-8")
    # Parse output into a python dict.
    p = re.compile('.+?precision')
    m = p.search(output)
    start = m.start()
    metricStr = output[start:].strip()
    metricObject = ast.literal_eval(metricStr)
    return [tup[1], tup[2], str(tup[0]), metricObject['precision']
                       ,metricObject['recall'],metricObject['accuracy'],metricObject['TP'],
                       metricObject['FP'], metricObject['FN']]

if regenerate:
    # Multiprocess method.
    classLabels = list(map(lambda item: item['id'], cats))
    argumentsList = []
    for model in modelNames:
        for transform in transformNames:
            for classLabel in classLabels:
                argumentsList.append([classLabel, model, transform])


    startTime = time.time()
    p = Pool(6)
    metricsList = p.map(f, argumentsList)
    endTime = time.time()
    print('Total time to run was %s seconds.'%(str(endTime-startTime)))

    # # Single process method.
    # classLabels = list(map(lambda item: item['id'], cats))
    # startTime = time.time()
    # for model in modelNames:
    #     for transform in transformNames:
    #         for i, classLabel in enumerate(classLabels):
    #             print('%s,%s,%s running...'%(model, transform, classLabel))
    #             result = subprocess.check_output(args=["python", "averageMetrics.py", "-c", str(classLabel), '-r',
    #                                      'basic', model, transform])
    #             output = result.decode("utf-8")
    #             # Parse output into a python dict.
    #             p = re.compile('.+?precision')
    #             m = p.search(output)
    #             start = m.start()
    #             metricStr = output[start:].strip()
    #             metricObject = ast.literal_eval(metricStr)
    #             metricsList.append([model, transform, classLabel, metricObject['precision']
    #                                ,metricObject['recall'],metricObject['accuracy'],metricObject['TP'],
    #                                metricObject['FP'], metricObject['FN']])

    # endTime = time.time()
    # print('Total time to run was %s seconds.'%(str(endTime-startTime)))

    # Build the pandas dataframe.
    metrics = pd.DataFrame(metricsList, columns=metricsColumns)
    # Save the pandas dataframe.
    metrics.to_csv('./metrics_dataframe.csv', index=True)





e2e_faster_rcnn_R-50-C4_2x,contrastnormalization_0,53 running...
e2e_faster_rcnn_R-50-C4_2x,None,1 running...
e2e_faster_rcnn_R-50-C4_2x,rotated_5,16 running...
e2e_faster_rcnn_R-101-FPN_2x,gaussianblur_20,86 running...
e2e_faster_rcnn_R-50-FPN_2x,translate_(0p1, -0p1),33 running...
e2e_faster_rcnn_R-50-FPN_2x,sharpen_0,70 running...
e2e_faster_rcnn_R-101-FPN_2x,gaussianblur_20,87 running...
e2e_faster_rcnn_R-50-C4_2x,contrastnormalization_0,54 running...
e2e_faster_rcnn_R-50-FPN_2x,translate_(0p1, -0p1),34 running...
e2e_faster_rcnn_R-50-C4_2x,rotated_5,17 running...
e2e_faster_rcnn_R-50-FPN_2x,sharpen_0,72 running...
e2e_faster_rcnn_R-50-C4_2x,None,2 running...
e2e_faster_rcnn_R-101-FPN_2x,gaussianblur_20,88 running...
e2e_faster_rcnn_R-50-FPN_2x,translate_(0p1, -0p1),35 running...
e2e_faster_rcnn_R-50-C4_2x,contrastnormalization_0,55 running...
e2e_faster_rcnn_R-50-C4_2x,rotated_5,18 running...
e2e_faster_rcnn_R-50-C4_2x,None,3 running...
e2e_faster_rcnn_R-50-FPN_2x,sharpen_0,73 run

e2e_faster_rcnn_R-50-FPN_2x,translate_(0p1, -0p1),58 running...
e2e_faster_rcnn_R-50-C4_2x,rotated_5,43 running...
e2e_faster_rcnn_R-101-FPN_2x,superpixels_0p1,22 running...
e2e_faster_rcnn_R-50-FPN_2x,sharpen_1,7 running...
e2e_faster_rcnn_R-50-C4_2x,None,27 running...
e2e_faster_rcnn_R-50-C4_2x,rotated_5,44 running...
e2e_faster_rcnn_R-50-C4_2x,contrastnormalization_0,82 running...
e2e_faster_rcnn_R-50-FPN_2x,translate_(0p1, -0p1),59 running...
e2e_faster_rcnn_R-101-FPN_2x,superpixels_0p1,23 running...
e2e_faster_rcnn_R-50-FPN_2x,sharpen_1,8 running...
e2e_faster_rcnn_R-50-C4_2x,rotated_5,46 running...
e2e_faster_rcnn_R-50-C4_2x,None,28 running...
e2e_faster_rcnn_R-50-C4_2x,contrastnormalization_0,84 running...
e2e_faster_rcnn_R-50-FPN_2x,translate_(0p1, -0p1),60 running...
e2e_faster_rcnn_R-101-FPN_2x,superpixels_0p1,24 running...
e2e_faster_rcnn_R-50-FPN_2x,sharpen_1,9 running...
e2e_faster_rcnn_R-50-C4_2x,rotated_5,47 running...
e2e_faster_rcnn_R-50-C4_2x,contrastnormalization_0,8

e2e_faster_rcnn_R-50-FPN_2x,sharpen_1,35 running...
e2e_faster_rcnn_R-50-C4_2x,None,53 running...
e2e_faster_rcnn_R-50-C4_2x,contrastnormalization_1,18 running...
e2e_faster_rcnn_R-50-FPN_2x,translate_(0p1, -0p1),88 running...
e2e_faster_rcnn_R-101-FPN_2x,superpixels_0p1,51 running...
e2e_faster_rcnn_R-50-C4_2x,rotated_5,73 running...
e2e_faster_rcnn_R-50-FPN_2x,sharpen_1,36 running...
e2e_faster_rcnn_R-50-C4_2x,None,54 running...
e2e_faster_rcnn_R-50-C4_2x,contrastnormalization_1,19 running...
e2e_faster_rcnn_R-101-FPN_2x,superpixels_0p1,52 running...
e2e_faster_rcnn_R-50-FPN_2x,translate_(0p1, -0p1),89 running...
e2e_faster_rcnn_R-50-C4_2x,rotated_5,74 running...
e2e_faster_rcnn_R-50-FPN_2x,sharpen_1,37 running...
e2e_faster_rcnn_R-50-C4_2x,None,55 running...
e2e_faster_rcnn_R-50-C4_2x,contrastnormalization_1,20 running...
e2e_faster_rcnn_R-101-FPN_2x,superpixels_0p1,53 running...
e2e_faster_rcnn_R-50-FPN_2x,translate_(0p1, -0p1),90 running...
e2e_faster_rcnn_R-50-C4_2x,rotated_5,75 

e2e_faster_rcnn_R-50-FPN_2x,translate_(-0p1, 0p1),23 running...
e2e_faster_rcnn_R-50-FPN_2x,sharpen_1,61 running...
e2e_faster_rcnn_R-50-C4_2x,rotated_10,8 running...
e2e_faster_rcnn_R-50-C4_2x,None,81 running...
e2e_faster_rcnn_R-101-FPN_2x,superpixels_0p1,80 running...
e2e_faster_rcnn_R-50-C4_2x,contrastnormalization_1,47 running...
e2e_faster_rcnn_R-50-FPN_2x,translate_(-0p1, 0p1),24 running...
e2e_faster_rcnn_R-50-C4_2x,rotated_10,9 running...
e2e_faster_rcnn_R-50-FPN_2x,sharpen_1,62 running...
e2e_faster_rcnn_R-50-C4_2x,None,82 running...
e2e_faster_rcnn_R-101-FPN_2x,superpixels_0p1,81 running...
e2e_faster_rcnn_R-50-C4_2x,contrastnormalization_1,48 running...
e2e_faster_rcnn_R-50-FPN_2x,translate_(-0p1, 0p1),25 running...
e2e_faster_rcnn_R-50-FPN_2x,sharpen_1,63 running...
e2e_faster_rcnn_R-50-C4_2x,rotated_10,10 running...
e2e_faster_rcnn_R-50-C4_2x,None,84 running...
e2e_faster_rcnn_R-101-FPN_2x,superpixels_0p1,82 running...
e2e_faster_rcnn_R-50-C4_2x,contrastnormalization_1,49

e2e_faster_rcnn_R-50-C4_2x,gaussianblur_1,17 running...
e2e_faster_rcnn_R-101-FPN_2x,superpixels_0p5,16 running...
e2e_faster_rcnn_R-50-C4_2x,rotated_10,36 running...
e2e_faster_rcnn_R-50-FPN_2x,translate_(-0p1, 0p1),51 running...
e2e_faster_rcnn_R-50-FPN_2x,sharpen_1,90 running...
e2e_faster_rcnn_R-50-C4_2x,contrastnormalization_1,75 running...
e2e_faster_rcnn_R-50-C4_2x,gaussianblur_1,18 running...
e2e_faster_rcnn_R-101-FPN_2x,superpixels_0p5,17 running...
e2e_faster_rcnn_R-50-C4_2x,rotated_10,37 running...
e2e_faster_rcnn_R-50-FPN_2x,translate_(-0p1, 0p1),52 running...
e2e_faster_rcnn_R-50-FPN_2x,sharpen_2,1 running...
e2e_faster_rcnn_R-50-C4_2x,contrastnormalization_1,76 running...
e2e_faster_rcnn_R-101-FPN_2x,superpixels_0p5,18 running...
e2e_faster_rcnn_R-50-C4_2x,gaussianblur_1,19 running...
e2e_faster_rcnn_R-50-C4_2x,rotated_10,38 running...
e2e_faster_rcnn_R-50-FPN_2x,translate_(-0p1, 0p1),53 running...
e2e_faster_rcnn_R-50-FPN_2x,sharpen_2,2 running...
e2e_faster_rcnn_R-50-C4

...

retinanet_X-101-64x4d-FPN_2x,flipV,51 running...
retinanet_R-101-FPN_2x,translate_(-0p1, -0p1),74 running...
retinanet_R-101-FPN_2x,addintensity_-80,19 running...
retinanet_X-101-64x4d-FPN_2x,scaled_0p5,89 running...
retinanet_R-50-FPN_2x,rotated_45,60 running...
retinanet_X-101-64x4d-FPN_2x,superpixels_0p85,41 running...
retinanet_X-101-64x4d-FPN_2x,flipV,52 running...
retinanet_R-101-FPN_2x,translate_(-0p1, -0p1),75 running...
retinanet_R-101-FPN_2x,addintensity_-80,20 running...
retinanet_R-50-FPN_2x,rotated_45,61 running...
retinanet_X-101-64x4d-FPN_2x,scaled_0p5,90 running...
retinanet_X-101-64x4d-FPN_2x,superpixels_0p85,42 running...
retinanet_X-101-64x4d-FPN_2x,flipV,53 running...
retinanet_R-101-FPN_2x,translate_(-0p1, -0p1),76 running...
retinanet_R-101-FPN_2x,addintensity_-80,21 running...
retinanet_R-50-FPN_2x,rotated_45,62 running...
retinanet_X-101-64x4d-FPN_2x,scaled_(1p25, 1p0),1 running...
retinanet_X-101-64x4d-FPN_2x,superpixels_0p85,43 running...
retinanet_X-101-64x4d

retinanet_R-101-FPN_2x,translate_(0p1, 0),10 running...
retinanet_X-101-64x4d-FPN_2x,superpixels_0p85,67 running...
retinanet_R-101-FPN_2x,addintensity_-80,48 running...
retinanet_X-101-64x4d-FPN_2x,flipV,81 running...
retinanet_R-50-FPN_2x,rotated_45,90 running...
retinanet_X-101-64x4d-FPN_2x,scaled_(1p25, 1p0),25 running...
retinanet_R-101-FPN_2x,translate_(0p1, 0),11 running...
retinanet_X-101-64x4d-FPN_2x,superpixels_0p85,70 running...
retinanet_R-101-FPN_2x,addintensity_-80,49 running...
retinanet_X-101-64x4d-FPN_2x,flipV,82 running...
retinanet_R-50-FPN_2x,rotated_60,1 running...
retinanet_X-101-64x4d-FPN_2x,scaled_(1p25, 1p0),27 running...
retinanet_R-101-FPN_2x,translate_(0p1, 0),13 running...
retinanet_X-101-64x4d-FPN_2x,flipV,84 running...
retinanet_X-101-64x4d-FPN_2x,superpixels_0p85,72 running...
retinanet_R-101-FPN_2x,addintensity_-80,50 running...
retinanet_R-50-FPN_2x,rotated_60,2 running...
retinanet_X-101-64x4d-FPN_2x,scaled_(1p25, 1p0),28 running...
retinanet_R-101-FP

retinanet_R-101-FPN_2x,addintensity_-80,77 running...
retinanet_X-101-64x4d-FPN_2x,colorspace_25,6 running...
retinanet_R-50-FPN_2x,rotated_60,25 running...
retinanet_X-101-64x4d-FPN_2x,dropout,19 running...
retinanet_R-101-FPN_2x,translate_(0p1, 0),40 running...
retinanet_R-101-FPN_2x,addintensity_-80,78 running...
retinanet_X-101-64x4d-FPN_2x,colorspace_25,7 running...
retinanet_X-101-64x4d-FPN_2x,scaled_(1p25, 1p0),54 running...
retinanet_R-50-FPN_2x,rotated_60,27 running...
retinanet_X-101-64x4d-FPN_2x,dropout,20 running...
retinanet_R-101-FPN_2x,translate_(0p1, 0),41 running...
retinanet_R-101-FPN_2x,addintensity_-80,79 running...
retinanet_X-101-64x4d-FPN_2x,scaled_(1p25, 1p0),55 running...
retinanet_X-101-64x4d-FPN_2x,colorspace_25,8 running...
retinanet_R-50-FPN_2x,rotated_60,28 running...
retinanet_X-101-64x4d-FPN_2x,dropout,21 running...
retinanet_R-101-FPN_2x,translate_(0p1, 0),42 running...
retinanet_X-101-64x4d-FPN_2x,colorspace_25,9 running...
retinanet_X-101-64x4d-FPN_2x

retinanet_X-101-64x4d-FPN_2x,dropout,48 running...
retinanet_R-101-FPN_2x,translate_(0p1, 0),67 running...
retinanet_R-50-FPN_2x,rotated_60,54 running...
retinanet_R-101-FPN_2x,addintensity_80,14 running...
retinanet_X-101-64x4d-FPN_2x,colorspace_25,36 running...
retinanet_X-101-64x4d-FPN_2x,scaled_(1p25, 1p0),84 running...
retinanet_X-101-64x4d-FPN_2x,dropout,49 running...
retinanet_R-101-FPN_2x,translate_(0p1, 0),70 running...
retinanet_R-50-FPN_2x,rotated_60,55 running...
retinanet_X-101-64x4d-FPN_2x,colorspace_25,37 running...
retinanet_R-101-FPN_2x,addintensity_80,15 running...
retinanet_R-101-FPN_2x,translate_(0p1, 0),72 running...
retinanet_X-101-64x4d-FPN_2x,scaled_(1p25, 1p0),85 running...
retinanet_X-101-64x4d-FPN_2x,dropout,50 running...
retinanet_X-101-64x4d-FPN_2x,colorspace_25,38 running...
retinanet_R-50-FPN_2x,rotated_60,56 running...
retinanet_R-101-FPN_2x,addintensity_80,16 running...
retinanet_R-101-FPN_2x,translate_(0p1, 0),73 running...
retinanet_X-101-64x4d-FPN_2x

retinanet_R-101-FPN_2x,addintensity_80,42 running...
retinanet_R-50-FPN_2x,rotated_60,84 running...
retinanet_R-101-FPN_2x,translate_(-0p1, 0),6 running...
retinanet_X-101-64x4d-FPN_2x,dropout,78 running...
retinanet_X-101-64x4d-FPN_2x,colorspace_25,63 running...
retinanet_R-101-FPN_2x,addintensity_80,43 running...
retinanet_X-101-64x4d-FPN_2x,scaled_(0p75, 1p0),20 running...
retinanet_R-50-FPN_2x,rotated_60,85 running...
retinanet_R-101-FPN_2x,translate_(-0p1, 0),7 running...
retinanet_X-101-64x4d-FPN_2x,dropout,79 running...
retinanet_X-101-64x4d-FPN_2x,colorspace_25,64 running...
retinanet_X-101-64x4d-FPN_2x,scaled_(0p75, 1p0),21 running...
retinanet_R-101-FPN_2x,addintensity_80,44 running...
retinanet_R-50-FPN_2x,rotated_60,86 running...
retinanet_R-101-FPN_2x,translate_(-0p1, 0),8 running...
retinanet_X-101-64x4d-FPN_2x,dropout,80 running...
retinanet_X-101-64x4d-FPN_2x,colorspace_25,65 running...
retinanet_X-101-64x4d-FPN_2x,scaled_(0p75, 1p0),22 running...
retinanet_R-101-FPN_2x

In [116]:
# Save the pandas dataframe.
pd.read_csv('./metrics_dataframe.csv')
metrics

,model,transform,class,precision,recall,accuracy,TP,FP,FN
0,e2e_faster_rcnn_R-50-C4_2x,None,1,0.694159,0.273512,0.244097,3009.731163,1326.064786,7994.268837
1,e2e_faster_rcnn_R-50-C4_2x,None,2,0.524687,0.156833,0.137323,49.559314,44.895615,266.440686
2,e2e_faster_rcnn_R-50-C4_2x,None,3,0.414337,0.120997,0.103325,233.765708,330.426081,1698.234292
3,e2e_faster_rcnn_R-50-C4_2x,None,4,0.719739,0.247889,0.226068,91.966830,35.811194,279.033170
4,e2e_faster_rcnn_R-50-C4_2x,None,5,0.779220,0.573851,0.493597,82.060714,23.250596,60.939286
5,e2e_faster_rcnn_R-50-C4_2x,None,6,0.745065,0.300459,0.272450,85.630862,29.299805,199.369138
6,e2e_faster_rcnn_R-50-C4_2x,None,7,0.852363,0.569802,0.518617,108.262338,18.751946,81.737662
7,e2e_faster_rcnn_R-50-C4_2x,None,8,0.486135,0.173240,0.146426,71.894444,75.995459,343.105556
8,e2e_faster_rcnn_R-50-C4_2x,None,9,0.530752,0.163732,0.143028,70.404960,62.246438,359.595040
9,e2e_faster_rcnn_R-50-C4_2x,None,10,0.271664,0.073914,0.061689,47.083333,126.231531,589.916667


In [103]:
import re


p = re.compile('.+?precision')
tmp="loading annotations into memory...\nDone (t=0.67s)\ncreating index...\nindex created!\n\nmodel:  e2e_faster_rcnn_R-50-C4_2x\ntransforms:  ['None']\nclasses:  [1]\n{'precision': 0.6941588576219956, 'recall': 0.273512464856328, 'accuracy': 0.2440969464012946, 'TP': 3009.7311632790147, 'FP': 1326.064786353547, 'FN': 7994.268836720916}\n"
#tmp="dfdfaprecision dkfjdf"
m = p.search(tmp) # Check each line
#if m: # If we have a match...
print(m.group()) # ...return the value


#start = tmp.find('[1]\n{\'precision\':')
#metricStr = tmp[start+4:].strip()
#metricObject = ast.literal_eval(metricStr)



{'precision
